In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array
import importlib

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')
folder_slope = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Slopes')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
folder_ela = os.path.join(folder_AGVA, 'Derived products', 'S2', 'ELAs')
rgis_to_analyze = rgis_folder # everything that is available

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})
rgis_to_analyze_df = rgis_to_analyze_df[rgis_to_analyze_df['Area']>=500]

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

22


In [5]:
out_df = rgis_to_analyze_df.copy()
out_df[['2018','2019','2020','2021','2022']] = 0
out_path = os.path.join(folder_save, 'Big Glacier Check', "first_look.csv")

if not os.path.exists(out_path):
    out_df.to_csv(out_path, index=False)

In [12]:
c=0
importlib.reload(snowFun)
for i in range(len(rgis_to_analyze)):
#     if i>0: continue
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
#     if rgiid != "RGI60-01.13696": continue
    
    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]
    
    if ga<500: continue
        
    c+=1
#     if c>1: continue
        
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2") 
    
    # create folder to save to if it doesn't exist
    folder_outputs = os.path.join(folder_save, 'Big Glacier Check', rgiid)
    if not os.path.exists(folder_outputs):
        os.makedirs(folder_outputs)
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry
    # single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier
    
    # define the coarsen scale
    if ga>1000:
        scale=5
    elif ga>500:
        scale=3
        
    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    if ga>500:
        glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8') 
#     print(glacier_mask)

    # get max elevation
    ele_max = snowFun.get_year_DEM(single_geometry, 2013, smoothed=0)
    ele_max = ele_max.max()
    
    # open the full ela dataframe
    path_open = os.path.join(folder_ela, f"{rgiid}_ELAs.csv")
    ela_df = pd.read_csv(path_open)
    
    # throw out anything that has <XYZ% observed
    ela_df_good = ela_df[ela_df['observed_after_smoothing']>0.7]
    ela_df_good = ela_df_good[ela_df_good['observed_initial']>0.3]
    
    # open the originally IDed annual best data
    path_open = os.path.join(folder_save, 'Annual AAs', 'csv', f"S2_{rgiid}_2018_2022_annual_AAs.csv")
    ela_df_best = pd.read_csv(path_open)
    
    # add year, month, day of year to each df
    ela_df_best['year'] = [ int(d[:4]) for d in ela_df_best['date'] ]
    ela_df_best['month'] = [ int(d[5:7]) for d in ela_df_best['date'] ]
    ela_df_best['doy'] = [ int(datetime.strptime(d, "%Y-%m-%d").timetuple().tm_yday) for d in ela_df_best['date'] ]
    ela_df_good['year'] = [ int(d[:4]) for d in ela_df_good['date'] ]
    ela_df_good['month'] = [ int(d[5:7]) for d in ela_df_good['date'] ]
    ela_df_good['doy'] = [ int(datetime.strptime(d, "%Y-%m-%d").timetuple().tm_yday) for d in ela_df_good['date'] ]
    ela_df_good['date_dt'] = pd.to_datetime(ela_df_good['date'])
    
#     for each year open the best image and save it
    for y in [2018,2019,2020,2021,2022]: #2018,2019,2020,2021,2022
        print(y)
        
        # subset each df to this year
        best_subset = ela_df_best[ela_df_best['year']==y]
        good_subset = ela_df_good[ela_df_good['year']==y]
        
        # get best date
        best_date = best_subset['date'].values[0]
        
        # also subset to the 15 days before/after the 'best date'
        doy = best_subset['doy'].values[0]
        good_subset_subset = good_subset[(good_subset['doy']<=(doy+15)) & (good_subset['doy']>=(doy-15))]
        
        # option to override and get just a specific date range
#         doy_range = [214,244]
#         year_exact = 2019
        
        if doy_range:
            if year_exact!=y: continue
            good_subset_subset = good_subset[(good_subset['doy']<=doy_range[1]) & (good_subset['doy']>=doy_range[0])]

        # open the final smoothed snow dataset
        path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed_coarse_smoothed.nc")
        snow = xr.open_dataset(path_open, chunks={'time':1, 'y':-1, 'x':-1})['class'].squeeze()
        
        # now for each product in the good_subset_subset, make figure to save
        for this_date in good_subset_subset['date'].values:
        
#             # grab the best data
#             best_snow = snow.sel(time=best_date)

            # grab the snow from this date
            this_snow = snow.sel(time=this_date)

            # fill in missing on-glacier data
    #         print(glacier_mask)
            glacier_mask = glacier_mask.reindex_like(this_snow, method='nearest')
            this_snow = xr.where( (glacier_mask==0) & (this_snow==0), -1, this_snow )

            ### make a figure to save
            fig,axs=plt.subplots(2,1, figsize=(20,20), height_ratios=[4, 1])
            axs[0].imshow(this_snow.values, vmin=-1, vmax=2, cmap='RdPu')
            axs[0].contour(glacier_mask.values, levels=[0.5], colors='black')
            axs[0].axis('off')
            axs[0].set_title(this_date, size=7)

            axs[1].scatter(good_subset['date_dt'],good_subset['ela'])
            axs[1].vlines([this_date], ymin=0, ymax=ele_max)
            axs[1].grid()
            axs[1].set_ylim(0, ele_max)

            plt.tight_layout()

            if this_date == best_date:
                name = f"{rgiid}_{this_date}_BEST.png"
            else:
                name = f"{rgiid}_{this_date}.png"
            plt.savefig(os.path.join(folder_outputs, name))
            plt.close()

print("Done!")
        


Starting 22 of 22: RGI60-01.13696  3362.656 km2
2018
2019
2020
2021
2022
Done!


In [ ]:
fig,axs=plt.subplots(2,1, figsize=(20,20), height_ratios=[4, 1])
axs[0].imshow(best_snow.values, vmin=-1, vmax=2, cmap='RdPu')
axs[0].contour(glacier_mask.values, levels=[0.5], colors='black')
axs[0].axis('off')
axs[0].set_title(best_date, size=7)

axs[1].scatter(good_subset['date_dt'],good_subset['ela'])
axs[1].vlines([best_date], ymin=0, ymax=np.nanmax(good_subset['ela']))

axs[1].grid()

plt.tight_layout()

name = f"{rgiid}_{best_date}.png"
# plt.savefig(os.path.join(folder_save, 'Big Glacier Check', name))

In [ ]:
print(xr.where( (glacier_mask==0) & (best_snow==0), -1, best_snow ))
print()
print(snow)